# Occupancy Forecasting Notebook

In [106]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from sklearn.model_selection import train_test_split, KFold

In [107]:
## Importing the dataset
#file_name = "frequency_data"
#data = pd.read_csv(f"data/cleaned_data/{file_name}.csv")

#dtypes = pd.read_csv(f"data/cleaned_data/{file_name}_dtypes.csv", index_col=0)
#data = data.astype(dtypes.to_dict()["0"])

#data["datetime"] = data["time"]
#data.drop("time", axis=1, inplace=True)


### Data Utility Functions

In [108]:
def filter_room_id(data, room_id):
    return data[data["room_id"] == room_id].reset_index(drop=True)

def filter_event_type(data, event_type):
    return data[data["event_type"] == event_type].reset_index(drop=True)

def resample(data, time_column, frequency, agg_func, output_columns=["event_type"]):
    
    # get min day
    if frequency == "MS":
        min_time = min(data[time_column]).replace(day=1, hour=0, minute=0, second=0)
    else:
        min_time = min(data[time_column]).replace(hour=0, minute=0, second=0)

    # get max day
    max_time = max(data[time_column])
    max_time = max_time.replace(day=max_time.day+1, hour=0, minute=0, second=0)
    
    
    idx = pd.date_range(start=min_time, end=max_time, freq=frequency, inclusive="both") 

    df_resampled = data.set_index(time_column)\
                   .resample(frequency, label="left", closed="left")

    if agg_func == "sum":
        df_resampled = df_resampled.sum()
    elif agg_func == "count":
        df_resampled = df_resampled.count()
    elif agg_func == "value_counts":
        df_resampled = df_resampled.value_counts()
    else:
        raise ValueError("agg_func must be 'sum', 'count' or 'value_counts'")
    
    return df_resampled.reindex(idx, fill_value=0).reset_index().rename(columns={"index": time_column})[[time_column] + output_columns]
    
def derive_day(data):
    data["day"] = data["datetime"].dt.date
    return data

def derive_week(data):
    data["week"] = data["datetime"].dt.isocalendar().week
    return data

def derive_time(data):
    data["time"] = data["datetime"].dt.time
    return data

def derive_weekday(data):
    data["weekday"] = data["datetime"].dt.weekday
    return data

In [137]:
def derive_in_out(dataframe):
    
    dataframe["people_in"] = dataframe["event_type"].apply(lambda x: 1 if x == 1 else 0)
    dataframe["people_out"] = dataframe["event_type"].apply(lambda x: 1 if x == 0 else 0)
    
    return dataframe

def calc_inside(dataframe):

    # calc people entering and leaving
    dataframe["people_inside"] = dataframe["people_in"].cumsum() - dataframe["people_out"].cumsum()
    
    return dataframe

def calc_occupancy_count(dataframe, time_column, frequency):
    
    df = dataframe.copy()
    
    df = derive_in_out(df)
    
    df_resampled = resample(df, time_column, frequency, "sum", output_columns=["people_in", "people_out"])
    
    df_resampled = calc_inside(df_resampled)
    
    return df_resampled
    

In [138]:
#data_filterd = filter_room_id(data, 0)
#calc_occupancy_count(data_filterd, "datetime", "1h")

,datetime,people_in,people_out,people_inside
0,2024-04-08 00:00:00,0,0,0
1,2024-04-08 01:00:00,0,0,0
2,2024-04-08 02:00:00,0,0,0
3,2024-04-08 03:00:00,0,0,0
4,2024-04-08 04:00:00,0,0,0
...,...,...,...,...
1964,2024-06-28 20:00:00,2,0,-241
1965,2024-06-28 21:00:00,0,0,-241
1966,2024-06-28 22:00:00,0,0,-241
1967,2024-06-28 23:00:00,0,0,-241


In [139]:
#data_filterd = filter_room_id(data, 0)
#data_filterd = filter_event_type(data_filterd, 1)

## 0. Basic Statistics and Plots

In [147]:
#data_filterd = filter_room_id(data, 0)

### Line Plot

In [148]:
#df_plot = calc_occupancy_count(data_filterd, "datetime", "15min")
#print(df_plot)
#fig = go.Figure()
## line plot
#fig.add_trace(
#    go.Scatter(
#        x=df_plot["datetime"], 
#        y=df_plot["people_inside"],
#        mode="lines",
#        name="people inside"))


Bar Charts

In [5]:
# Paper: 
# Basic bar plots count of events after resampling
# Check if it follows some distribution

data_filterd = filter_event_type(data_filterd, 1)
data_resampled = resample(data_filterd, "datetime", "5min", "count")


data_trimmed = data_resampled[data_resampled["event_type"] > 0]
value_counts = data_trimmed.value_counts("event_type")

fig = go.Figure()
# line plot
fig.add_trace(
    go.Bar(
        x=value_counts.index, 
        y=value_counts, 
        name='value_counts'))


Boxplot

In [6]:
# Boxplot plots -> Boxplot for every day and time
# Do for different events and rooms
data_resampled = resample(data_filterd, "datetime", "15min", "count")
data_resampled = derive_time(data_resampled)
data_resampled = derive_weekday(data_resampled)
data_resampled = derive_week(data_resampled)
data_resampled = data_resampled.drop("datetime", axis=1)


data_plot = data_resampled.groupby(["week", "time", "weekday"]).sum().reset_index()
data_plot = data_plot[data_plot["weekday"] == 0]

print(data_plot)
fig = px.box(data_plot, x="time", y="event_type")
fig.show()

      week      time  weekday  event_type
0       15  00:00:00        0           0
7       15  00:15:00        0           0
14      15  00:30:00        0           0
21      15  00:45:00        0           0
28      15  01:00:00        0           0
...    ...       ...      ...         ...
7848    26  22:45:00        0           0
7853    26  23:00:00        0           0
7858    26  23:15:00        0           0
7863    26  23:30:00        0           0
7868    26  23:45:00        0           0

[1152 rows x 4 columns]


More plots in the other notebook

## 1.Patterns

clustering -> different algorithms, k-means

Papers tried: 

### 1.1 Occupancy data analytics and prediction: A case study - Xin Liang a, b, Tianzhen Hong b, *, Geoffrey Qiping Shen

Distance Measures: Eucleadian, Dynamic Time Warp, Correlation Simmilarity -> compared among all different configurations

Clustering Algos: K-Means, 

Algorithm Evaluation: Davies-Bouldin index, to find best k

Insights of the paper:

* High variability in the data statistical methods using some kind of mean will fail miserably!

In [7]:
import json
from collections import defaultdict
import itertools
from tqdm import tqdm


In [8]:
def filter_by_time(data, time_column, start_time, end_time):
    
    return data[(data[time_column].dt.time >= start_time) & (data[time_column].dt.time <= end_time)]

def plot_cluster_details(cluster_number, clustering_model, samples, time_axis, plot_centers):
    
    indices = np.arange(len(samples))
    
    fig = go.Figure()
    
    cluster_indices = indices[clustering_model.labels_ == cluster_number]
    
    if plot_centers:
        cluster_center = clustering_model.cluster_centers_[cluster_number]
        fig.add_trace(
            go.Scatter(
                x=time_axis, 
                y=cluster_center, 
                name=f'Center',
                mode="lines+markers"        
            )
        )

    
    print(f"################ {cluster_number} ################")
    print(cluster_indices)

    for x in range(len(cluster_indices)):
        fig.add_trace(
            go.Scatter(
                x=time_axis, 
                y=samples[cluster_indices[x]], 
                mode="lines+markers"        
            )
        )
    
    return fig

def plot_cluster_centers(n_clusters, clustering_model, times):
    
    fig = go.Figure()
    
    for i in range(n_clusters):

        cluster_center = clustering_model.cluster_centers_[i]
        
        # line plot
        fig.add_trace(
            go.Scatter(
                x=times, 
                y=cluster_center, 
                name=f'cluster {i}',
                mode="lines+markers"        
            )
        )
        
    return fig

# copied from signal_processing/utils.py and changed a bit
class ParameterSearch:
    
    def __init__(self, path_to_json=None, parameter_dict=None):
        
        # check if either path_to_parameters or parameter_dict is given
        if path_to_json is None and parameter_dict is None:
            raise ValueError("Either path_to_parameters or parameter_dict must be given")
        # check if both are given
        elif path_to_json is not None and parameter_dict is not None:
            raise ValueError("Either path_to_parameters or parameter_dict must be given")
        # check if path_to_json is given
        elif path_to_json is not None:
            # read json file
            with open(path_to_json, "r") as file:
                self.parameter_dict = json.load(file)
        # check if parameter_dict is given
        elif parameter_dict is not None:
            self.parameter_dict = parameter_dict
        else:
            raise ValueError("Something went wrong!")
          
    #### Generate all possible combinations of parameters ####
    def _extract_all_combinations(self, params_dict):
    
        values = []
        keys = []
        for key in params_dict.keys():
            
            # implement check if current entry is a dictionary
            sub_dict = params_dict[key]
            
            for sub_key in sub_dict.keys():
                values.append(sub_dict[sub_key])
                keys.append((key, sub_key))
        
        n_combinations = np.prod([len(value) for value in values])

        return keys, iter(itertools.product(*values)), n_combinations
     
    def _generate_comb_dict(self, keys, combination):
        dictionary = defaultdict(dict)
        for key, value in zip(keys, combination):
                dictionary[key[0]][key[1]] = value
        return dictionary
        
    def _multi_combinations_wrapper(self, params_dict, tqdm_bar):
        
        keys, comb_iterator, n_combs = self._extract_all_combinations(params_dict=params_dict)
        
        if tqdm_bar:
            tqdm_iterator = tqdm(comb_iterator, total=n_combs)
            for x in tqdm_iterator:
                yield self._generate_comb_dict(keys, x)
        else:
            for x in comb_iterator:
                yield self._generate_comb_dict(keys, x)
    
    def mulit_function_combinations_iterator(self, tqdm_bar=True):
        # This method is used for the user to iterate over all possible combinations of multiple functions
        # Needs special structure of parameter dictionaries
        return self._multi_combinations_wrapper(params_dict=self.parameter_dict, tqdm_bar=tqdm_bar)
    
    #### Simple Grid Search ####
    def _extract_all_combinations_simple(self, params_dict):
        
        values = []
        keys = []
        for key in params_dict.keys():
            values.append(params_dict[key])
            keys.append(key)
        
        n_combinations = np.prod([len(value) for value in values])

        return keys, iter(itertools.product(*values)), n_combinations
    
    def _grid_search_wrapper(self, params_dict, tqdm_bar):
        
        keys, comb_iterator, n_combs = self._extract_all_combinations_simple(params_dict=params_dict)
        
        if tqdm_bar:
            tqdm_iterator = tqdm(comb_iterator, total=n_combs)
            for x in tqdm_iterator:
                yield dict(zip(keys, x))
        else:
            for x in comb_iterator:
                yield dict(zip(keys, x))
                
    def grid_search_iterator(self, tqdm_bar=True):
        return self._grid_search_wrapper(params_dict=self.parameter_dict, tqdm_bar=tqdm_bar)
    

In [9]:
# split into daily samples
data_resampled = resample(data_filterd, "datetime", "15min", "count")
data_resampled = derive_time(data_resampled)
data_resampled = derive_weekday(data_resampled)
data_resampled = derive_week(data_resampled)
data_resampled = derive_day(data_resampled)

data_resampled = filter_by_time(data_resampled, "datetime", datetime.strptime("08:00:00", "%H:%M:%S").time(), datetime.strptime("20:00:00", "%H:%M:%S").time())


samples = []
times = []
group_list = []
for group in data_resampled.groupby("day"):
    #print(group[1]["event_type"].reset_index(drop=True))
    samples.append(group[1]["event_type"].values)
    times.append(group[1]["datetime"].values)
    group_list.append(group[0])
    
samples = np.array(samples[:-1])
times = np.array(times[:-1])
group_list = np.array(group_list[:-1])


In [12]:
from sklearn.cluster import KMeans, AffinityPropagation, MeanShift, SpectralClustering, AgglomerativeClustering
from sklearn.metrics import davies_bouldin_score, pairwise_distances

from scipy.spatial.distance import pdist, squareform



Distance metrics:

* Euclidean
* Correlation Similarity
* Dynamic Time Warp



In [38]:
# get optimal configuration

from sklearn.cluster import DBSCAN, BisectingKMeans
from sklearn.manifold import TSNE

seeds = range(0,100)

parameters = {
    "general_settings":{},
    "downprojection": False,
    "algorithm":"kmeans",
    "kmeans":{
        "n_clusters": list(range(2, 15)),
    },
    "bisectingkmeans":{
        "n_clusters": list(range(2, 15)),
        "n_init": [1, 5, 10, 20],
    },
    "affinity":{
        "damping": np.linspace(0.5, 1, 10, endpoint=False),
        "affinity": ["euclidean"],
    },
    "dbscan":{
        "eps": np.linspace(0.1, 1, 10),
        "min_samples": list(range(5, 15)),
        "metric": ["euclidean", "manhattan", "correlation"],
        "leaf_size": [2,5,10,30],
    },
    "meanshift":{
        },
    "spectral":{
       "n_clusters": list(range(2, 15)), 
    },
    "agglomerative":{
        "n_clusters": list(range(2, 15)),
        "metric": ["manhattan", "cosine", "l1", "l2"],
        "linkage": ["complete", "average", "single"],
    },
    
    "tsne":{
        "n_components": [3],
        "perplexity": [30],
    }
}


if parameters["algorithm"] == "kmeans":
    algorithm_instance = KMeans
elif parameters["algorithm"] == "bisectingkmeans":
    algorithm_instance = BisectingKMeans
elif parameters["algorithm"] == "affinity":
    algorithm_instance = AffinityPropagation
elif parameters["algorithm"] == "dbscan":
    algorithm_instance = DBSCAN
elif parameters["algorithm"] == "meanshift":
    algorithm_instance = MeanShift
elif parameters["algorithm"] == "spectral":
    algorithm_instance = SpectralClustering
elif parameters["algorithm"] == "agglomerative":
    algorithm_instance = AgglomerativeClustering
else:
    raise ValueError("Algorithm not implemented!")

argmin_list = []
for seed in seeds:
    
    bdi_scores = []
    combination = []
    comb_iterator = ParameterSearch(parameter_dict=parameters[parameters["algorithm"]]).grid_search_iterator(tqdm_bar=True)

    for i, params in enumerate(comb_iterator):
        
        #print(f"################ {n_clusters} ################")
        
        clustering_model = algorithm_instance(**params)
        
        #comb_iterator_tsne = ParameterSearch(parameter_dict=parameters["tsne"]).grid_search_iterator(tqdm_bar=False)
        
        
        #for params_tsne in comb_iterator_tsne:
        #    if parameters["downprojection"]:
        #        samples_embedded = TSNE(**params_tsne).fit_transform(samples)
            
        samples_embedded = samples
        try:
            if params["affinity"] == "precomputed":
                similarities = - pairwise_distances(samples_embedded + 1e-8, metric="manhattan")
                clustering_model.fit(similarities)
            else:
                clustering_model.fit(samples_embedded)
        except:
            clustering_model.fit(samples_embedded + 1e-8)


        #plot_cluster_centers(n_clusters, clustering_model, times[0]).show()
        if len(np.unique(clustering_model.labels_)) == 1:
            score = 1000
        else:
            score = davies_bouldin_score(samples, clustering_model.labels_)

        #print(score)
        bdi_scores.append(np.round(score, 6))
        #combination.append((params, params_tsne))
        combination.append((i, params))

        
    argmin_list.append(np.argmin(bdi_scores))
np.unique(argmin_list, return_counts=True)

100%|██████████| 13/13 [00:00<00:00, 226.09it/s]


(array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([ 5,  1,  1,  2,  2,  2,  3,  6,  7, 15, 21, 35]))

### Optimal Parameters

algorithm = KMeans , INVESTIGATE FURTHER -- Read the two papers and implement their approach<br>
include_seeds = True <br>
algo_parameters = {
    "n_clusters":12
}

BisectingKMeans, INVESTIGATE FURTHER -- we need seeds<br>
{'n_clusters': 14, 'n_init': 20}

algorithm = AffinityPropagation -> works well, GRAPHCLUSTERING<br>
{'damping': 0.5, 'affinity': 'euclidean'}

AgglomerativeClustering -> works well <br>
{'n_clusters': 2, 'metric': 'manhattan', 'linkage': 'average'}

Fuzzy C-Means <br>
Implementation is flawed

DBScan <br>
Figure out better parameters


Dimensionality reduction:
* TSNE enhanced Clustering:

No TSNE<br>
{'n_clusters': 14}
Number of clusters: 14
BDI: 0.7672581861245025

* UMAP enahnced Clustering


In [39]:
combination

[(0, {'n_clusters': 2}),
 (1, {'n_clusters': 3}),
 (2, {'n_clusters': 4}),
 (3, {'n_clusters': 5}),
 (4, {'n_clusters': 6}),
 (5, {'n_clusters': 7}),
 (6, {'n_clusters': 8}),
 (7, {'n_clusters': 9}),
 (8, {'n_clusters': 10}),
 (9, {'n_clusters': 11}),
 (10, {'n_clusters': 12}),
 (11, {'n_clusters': 13}),
 (12, {'n_clusters': 14})]

In [40]:
# test optimal configuration
print(f"################ {parameters['algorithm']} ################")
optimal_params = combination[10][1]
print(optimal_params)

clustering_model = algorithm_instance(**optimal_params)

try:
    if params["affinity"] == "precomputed":
        similarities = - pairwise_distances(samples + 1e-8, metric="manhattan")
        clustering_model.fit(similarities)
    else:
        clustering_model.fit(samples)
except:
    clustering_model.fit(samples)

try:
    n_clusters = len(clustering_model.cluster_centers_)
except:
    try:
        clustering_model.cluster_centers_ = samples[clustering_model.cluster_centers_indices_]
        n_clusters = len(clustering_model.cluster_centers_indices_)
    except:
        n_clusters = len(np.unique(clustering_model.labels_))
    
    

print(f"Number of clusters: {n_clusters}")

#plot_cluster_centers(n_clusters, clustering_model, times[0]).show()

score = davies_bouldin_score(samples, clustering_model.labels_)
print(f"BDI: {score}")

################ kmeans ################
{'n_clusters': 12}
Number of clusters: 12
BDI: 0.809285789249472


In [41]:
for i in range(n_clusters):  
    plot_cluster_details(i, clustering_model, samples, times[0], plot_centers=False).show()

################ 0 ################
[ 0  7 14 21]


################ 1 ################
[ 6 12 13 19 20 23 25 26 27 28 31 32 33 34 40 41 42 43 46 47 48 50 52 53
 54 55 60 61 62 64 67 68 69 74 75 76 77]


################ 2 ################
[ 5  8 10 17 18 24 37 38 44 45 58 59 66 72 73 79]


################ 3 ################
[78]


################ 4 ################
[80]


...

################ 7 ################
[22]


################ 8 ################
[ 2  9 16 30 51 57 65]


################ 9 ################
[11 35 49 56 63 70]


################ 10 ################
[3]


################ 11 ################
[ 1 15 36 71]


In [20]:
indices = np.arange(len(samples))

fig = go.Figure()

for i in range(n_clusters):

    cluster_indices = indices[clustering_model.labels_ == i]
    cluster_center = clustering_model.cluster_centers_[i]
    
    
    cluster_group_list = group_list[cluster_indices]

    print(f"################ {i} ################")
    print(cluster_indices)
    print(cluster_group_list)
    print()
    
    # line plot
    fig.add_trace(
        go.Scatter(
            x=times[0], 
            y=cluster_center, 
            name=f'cluster {i}',
            mode="lines+markers"        
        )
    )
        
fig.show()

################ 0 ################
[ 6 12 13 19 20 23 25 26 27 31 32 33 34 40 41 42 43 46 47 48 52 53 54 55
 60 61 62 67 68 69 74 75 76]
[datetime.date(2024, 4, 14) datetime.date(2024, 4, 20)
 datetime.date(2024, 4, 21) datetime.date(2024, 4, 27)
 datetime.date(2024, 4, 28) datetime.date(2024, 5, 1)
 datetime.date(2024, 5, 3) datetime.date(2024, 5, 4)
 datetime.date(2024, 5, 5) datetime.date(2024, 5, 9)
 datetime.date(2024, 5, 10) datetime.date(2024, 5, 11)
 datetime.date(2024, 5, 12) datetime.date(2024, 5, 18)
 datetime.date(2024, 5, 19) datetime.date(2024, 5, 20)
 datetime.date(2024, 5, 21) datetime.date(2024, 5, 24)
 datetime.date(2024, 5, 25) datetime.date(2024, 5, 26)
 datetime.date(2024, 5, 30) datetime.date(2024, 5, 31)
 datetime.date(2024, 6, 1) datetime.date(2024, 6, 2)
 datetime.date(2024, 6, 7) datetime.date(2024, 6, 8)
 datetime.date(2024, 6, 9) datetime.date(2024, 6, 14)
 datetime.date(2024, 6, 15) datetime.date(2024, 6, 16)
 datetime.date(2024, 6, 21) datetime.date(2024,

## 2. Train and Test split

In [6]:
data_filterd = filter_room_id(data, 0)
data_filterd = filter_event_type(data_filterd, 1)


#train, test = train_test_split(samples, test_size=0.1, random_state=42)

#train = np.array(train)
#test = np.array(test)

#kf = KFold(n_splits=10)

#for train_index, valid_index in kf.split(train):
    
#    train_fold = train[train_index]
#    valid_fold = train[valid_index]
    
#    break

## 3. Occupancy Forecasting

In [ ]:
# check out some of the papers in the review
# start with some traditional methods

## 4. Count Forecasting
### Try more general approach if 1. is too hard, gather literature on count forecasting